In [ ]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklvq import GMLVQ
from timeit import timeit
from sklearn.model_selection import KFold
from itertools import chain
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import (
    cross_val_score,
    RepeatedKFold,
)
from sklearn.pipeline import make_pipeline
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve,roc_auc_score,auc
import colorama
from colorama import Fore, Style
matplotlib.rc("xtick", labelsize="small")
matplotlib.rc("ytick", labelsize="small")

In [ ]:
def getdata():
    data = pd.read_csv("feature_vectors.csv").to_numpy()
    labels = pd.read_csv("diagnosis_label.csv").to_numpy().squeeze()
    labelscenter = pd.read_csv("center_label.csv").to_numpy().squeeze()

    labelsfinal = labels + labelscenter
    
    # better way to perform this
    labelsdiseases1 = np.where(labelsfinal == 'HCUMCG') 
    centerlabels1 = labelsfinal[labelsdiseases1]
    labelsdiseases2 = np.where(labelsfinal == 'HCUGOSM') 
    #labelsdiseases = np.where(labels == 'HCCUN','1',centerlabels) 
    centerlabels2 = labelsfinal[labelsdiseases2]
    labelsdiseases3 = np.where(labelsfinal == 'HCCUN') 
    centerlabels3 = labelsfinal[labelsdiseases3]
    
    # better way to perform this
    centerlabels = np.concatenate((centerlabels1,centerlabels2,centerlabels3))
    centerdata = data[labelsdiseases1]
    centerdata = np.concatenate((centerdata,data[labelsdiseases2]))
    centerdata = np.concatenate((centerdata,data[labelsdiseases3]))
    
    scalar = StandardScaler()
    #data = ztransform(data,scaler)
    #centerdata= ztransform(centerdata,scaler)
    return data,labels,centerdata,centerlabels,labelscenter,scalar

In [ ]:
def ztransform(data_ztransform,scalarz):
    # Sklearn's standardscaler to perform z-transform

    # Compute (fit) and apply (transform) z-transform
    # why do we do an inverse z transform
    #scalar = scalar
    data_ztransform = scalarz.fit_transform(data_ztransform)
    
    return data_ztransform,scalarz

In [ ]:
def model_definition_center():
    
    model = GMLVQ(
    distance_type="adaptive-squared-euclidean",
    activation_type="sigmoid",
    activation_params={"beta": 2},
        #wgd
    solver_type="wgd",
    solver_params={"max_runs": 300,"step_size": np.array([0.1, 0.05])},
    #solver_params={"max_runs": 10,"batch_size":1,"step_size": np.array([0.1, 0.05])},
    random_state=1428,)
    
    return model

In [ ]:
def model_definition_disease(correctionmatrix):
    model1 = GMLVQ(
        distance_type="adaptive-squared-euclidean",
        activation_type="sigmoid",
        activation_params={"beta": 2},
        solver_type="wgd",
        solver_params={"max_runs": 150,"step_size": np.array([0.1, 0.05])},
        #solver_params={"max_runs": 10,"batch_size":1,"step_size": np.array([0.1, 0.05])},
        random_state=1428,
        relevance_correction=correctionmatrix
    )
    return model1

In [ ]:
run the whole notebook for max runs say(1-10000) # check exactly what does max runs define in this. calculate the loss and store at each max runs step

In [ ]:
def train_modelkfold(data, label,disease,correctionmatrix1,repeated,scalar,folds=5):
    modelmatrix = np.zeros((repeated,folds),dtype=object) 
    scalars = np.zeros((repeated,folds),dtype=object)
    datamatrix =  np.zeros((repeated,folds),dtype=object) 
    labelmatrix = np.zeros((repeated,folds),dtype=object)
    accuracies = np.zeros((repeated,folds),dtype=float) 
    # each entry will store the test labels and the predicted labels
    testlabels = np.zeros((repeated,folds),dtype=object) 
    predicted = np.zeros((repeated,folds),dtype=object) 
    probablities = np.zeros((repeated,folds),dtype=object) 
    laeblscore = np.zeros((repeated,folds),dtype=object) 
    testing_indicesM = np.zeros((repeated,folds),dtype=object) 
    traning_indicesM = np.zeros((repeated,folds),dtype=object) 
    
    if disease==False:
        print('Repeated K fold for center data')
    else:
        print('Repeated K fold for disease data')
    for repeated in range(repeated):
        
        print("========Repeated fold number",str(repeated),"========")
        kfold = KFold(folds, shuffle=True)
        
        for k, (training_indices, testing_indices) in enumerate(kfold.split(data)):
            if disease==False:
                model = model_definition_center()
                #print('in this')
                #print(type(model))
            else:
                #model = model_definition_center()
                model = model_definition_disease(correctionmatrix1)
            accuracy = 0
            correct = 0
            trainX, trainY, testX, testY = data[training_indices], label[training_indices], data[testing_indices], label[testing_indices]
            #applying z transform on training data only
            trainX,scalar1 = ztransform(trainX,scalar)
            model.fit(trainX,trainY)
            
            # model is fitted, can also do = model
            modelmatrix[repeated,k] = model
            datamatrix[repeated,k] = trainX
            
            
            labelmatrix[repeated,k] = trainY

            testX = scalar1.transform(testX)
            predictedlabels = model.predict(testX)
            #print(confusion_matrix(testX, predictedlabels))
            for i in range(len(predictedlabels)):
                if(predictedlabels[i]==testY[i]):
                    correct = correct+1
                    
            scalars[repeated,k] = scalar1
            
            probabilties = model.predict_proba(testX)
            
            laeblscore[repeated,k] = model.decision_function(testX)
            
            probablities[repeated,k] = probabilties
            
            testlabels[repeated,k] = testY
            
            predicted[repeated,k] = predictedlabels
            
            
            testing_indicesM[repeated,k] = testing_indices
            traning_indicesM[repeated,k] = training_indices
            
            accuracy = correct/len(testY)
            print('%.3f accuracy' % (correct/len(testY)))
    return modelmatrix,datamatrix,labelmatrix,scalars,testlabels,predicted,probablities,laeblscore,testing_indicesM,traning_indicesM

In [ ]:
def calculate_lambda(modelmatrix,repeated):
    #average_lambda = np.mean(sum(model.lambda_ for model in modellist),axis=0)
    #check the shape of total_lambda
    
    total_lambda = np.zeros((36,36),dtype = 'float')
    print(repeated)
    for index in range(repeated):
        print(index)
        lambdaperfold = np.zeros((36,36),dtype='float')        
        
        for modelin in modelmatrix[index,:]:
            lambdaperfold = lambdaperfold + modelin.lambda_

        lambdaperfold = lambdaperfold/np.shape(modelmatrix)[1]
        
        total_lambda = total_lambda + lambdaperfold
        
    total_lambda = total_lambda/repeated

    return total_lambda

In [ ]:
def eigendecomposition(average_lambda):
    eigenvalues, eigenvectors = np.linalg.eigh(average_lambda)
    # eigen vectors returned in columns
    # Flip (reverse the order to descending) before assigning.
    eigenvalues = np.flip(eigenvalues)

    # eigenvectors are column matrix in ascending order. Flip the columns and transpose the matrix
    # to get the descending ordered row matrix.
    
    # eigenvectors are row matrices
    eigenvectors = np.flip(eigenvectors, axis=1).T
    
    return eigenvalues,eigenvectors

In [ ]:
feature_names = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z','ab','ac','ad','ae','af','ag','ah','ai','aj','ak']


In [ ]:

def correction_matrix(eigvectorscenter):
    N=36
    K=2
    I = np.identity(N)
    outerproduct = np.zeros((N,N))   
    for i in range(K):
        # check this
        outerproduct +=  np.outer(eigvectorscenter.T[:,i],eigvectorscenter[i,:])
        outerproduct1 =  np.outer(eigvectorscenter.T[:,0],eigvectorscenter[0,:])
        outerproduct2 =  np.outer(eigvectorscenter.T[:,1],eigvectorscenter[1,:])        
    correctionmatrix = I-outerproduct
    return correctionmatrix

In [ ]:
def calculate_prototype(modelmatrix,repeated):
    #average_lambda = np.mean(sum(model.lambda_ for model in modellist),axis=0)
    #check the shape of total_lambda
    # prototypes,number of dimension
    total_prototype = np.zeros((3,36),dtype = 'float')
    for index in range(repeated):
        prototypeperfold = np.zeros((3,36),dtype='float')        
        for modelin in modelmatrix[index,:]:
            for prototype in range(len(modelin.prototypes_)):
                prototypeperfold  = prototypeperfold  + modelin.prototypes_
        prototypeperfold = prototypeperfold/np.shape(modelmatrix)[1]
        
        total_prototype = total_prototype + prototypeperfold
        
    total_prototype = total_prototype/repeated

    return total_prototype

In [ ]:
def calculate_prototype1(modelmatrix,repeated):
    #average_lambda = np.mean(sum(model.lambda_ for model in modellist),axis=0)
    #check the shape of total_lambda
    
    total_prototype = np.zeros((5,36),dtype = 'float')
    for index in range(repeated):
        prototypeperfold = np.zeros((5,36),dtype='float')        
        for modelin in modelmatrix[index,:]:
            for prototype in range(len(modelin.prototypes_)):
                prototypeperfold  = prototypeperfold  + modelin.prototypes_
        prototypeperfold = prototypeperfold/np.shape(modelmatrix)[1]
        
        total_prototype = total_prototype + prototypeperfold
        
    total_prototype = total_prototype/repeated

    return total_prototype

## center data fitting (3 class)

In [ ]:
#data,labels,centerdata,centerlabels,labelscenter,scalar = getdata()
##################individual model######################################
data,labels,centerdata1,centerlabels1,labelscenter,scalar = getdata()

healthycontrolindices = np.where(labels=='HC')

datanoHC = np.delete(data,healthycontrolindices,axis=0)

labelsnoHC = np.delete(labels,healthycontrolindices,axis=0)

labelscenternoHC = np.delete(labelscenter,healthycontrolindices,axis=0)

labels4class = labelsnoHC + labelscenternoHC
#######################################################################

centermodel,datamatrix,labelmatrix,scalars1,testlabelsc,predictedc,probabiltiesc,laeblscorec,testing_indicesC,training_indicesC = train_modelkfold(centerdata1,centerlabels1,False,None,100,scalar)

average_lambda_center = calculate_lambda(centermodel,100)

average_prototype_center = calculate_prototype(centermodel,100)

print('                                                     ')


print('The sum of the digonal of the center relevance matrix')

print(np.sum(np.diagonal(average_lambda_center)))

print('                                                     ')


eigenvaluescenter,eigenvectorscenter = eigendecomposition(average_lambda_center)


correctionmatrix1 = correction_matrix(eigenvectorscenter)

#correctionmatrix = correction_matrix()


## Disease data fitting 4 class AD PD

In [ ]:

diseasemodel,datamatrix1,labelmatrix1,scalars2,testlabelsd,predictedd,probabiltiesd,laeblscored,testing_indicesD,training_indicesD  = train_modelkfold(datanoHC,labels4class,True,correctionmatrix1,2,scalar)

average_lambda_disease = calculate_lambda(diseasemodel,100)

eigenvaluesdisease,eigenvectorsdisease = eigendecomposition(average_lambda_disease)


average_prototype_disease = calculate_prototype1(diseasemodel,100)




print('The sum of the digonal of the disease relevance matrix')

print(np.sum(np.diagonal(average_lambda_disease)))


## single model trained on 4 class disease and 3 class center data

In [ ]:
# training an individual model

modelc = model_definition_center()
centerdata1,scalarIc = ztransform(centerdata1,scalar)

modelc.fit(centerdata1,centerlabels1)

correctionmatrix1 = correction_matrix(modelc.eigenvectors_)
datanoHC,scalarId = ztransform(datanoHC,scalar)

modeld = model_definition_center()

modeld.fit(datanoHC,labels4class)
predictedlabels = modeld.predict(datanoHC)

## Zero dot product

In [ ]:
# removed the influence of the leading eigen vector from each 
first_eigen_vector = np.dot(diseasemodel[0][3].eigenvectors_[0,:],eigenvectorscenter[1,:])
print(first_eigen_vector)

In [ ]:
def check_orthogonality(centermodel,diseasemodel,dimensions,repeated,folds,eigenvectorscenter):
    centerlist = list(chain.from_iterable(zip(*centermodel)))
    disease_center = list()
    disease_averagecenter= list()
    for j in range(2):
        for array,modeld in enumerate(diseasemodel[j]):
            disease_averagecenter.append(np.dot(modeld.eigenvectors_[:2,:],eigenvectorscenter[:2,:].T))
            for modelc in centerlist:
                # eigenvectors are row vectors
                disease_center.append((np.dot(modeld.eigenvectors_[:2,:],modelc.eigenvectors_[:2,:].T)))
    return disease_averagecenter,disease_center

## Visualize single model 4 class and 3 class center

In [ ]:
# removing the z transform
def visualizeSinglemodel(model,scalarI,data,labels,eigenvaluesaverage,eigenvectoraverage,average_prototype):
    centerdata = scaler.inverse_transform(centerdata)
    #transformed_data = transform1(centerdata,eigenvaluesdisease,eigenvectoraverage,True)

    #x_d = transformed_data[:, 0]
    #y_d = transformed_data[:, 1]

    # Transform the model, i.e., the prototypes (scaled by square root of eigenvalues "scale = True")
    # prototype inverser transform
    #prototypes = scaler.inverse_transform(model.prototypes_)
    
    #transformed_model = model.transform(model.prototypes_, scale=True)
    
    transformed_data = model.transform(centerdata,True)
    
    # all examples 1st and 2nd feature
    x_d = transformed_data[:, 0]
    y_d = transformed_data[:, 1]

    # Transform the model, i.e., the prototypes (scaled by square root of eigenvalues "scale = True")
    # prototype inverser transform
    prototypes = scaler.inverse_transform(model.prototypes_)

    transformed_model = model.transform(prototypes, scale=True)

    #transformed_model = transform1(prototypes,eigenvaluesaverage,eigenvectoraverage,scale=True)

    x_m = transformed_model[:, 0]
    y_m = transformed_model[:, 1]

    fig, ax = plt.subplots()
    fig.suptitle("Maindataset disease data with corresponding prototypes")
    colors = ['pink','magenta','brown','pink','lightgreen']
    for i, cls in enumerate(model.classes_):
        ii = cls == centerlabels
        ax.scatter(
            x_d[ii],
            y_d[ii],
            c=colors[i],
            s=100,
            alpha=0.7,
            edgecolors="white",
            label=model.classes_[model.prototypes_labels_[i]],
        )
    ax.scatter(x_m, y_m, c=colors, s=180, alpha=0.8, edgecolors="black", linewidth=2.0)
    ax.set_xlabel("First eigenvector")
    ax.set_ylabel("Second eigenvector")
    ax.legend()
    #plt.ylim([-1, 30])
    #plt.xlim([-30, 30])
    ax.grid(True)
    print(model.classes_)
    #fig.savefig('visualize_center.eps', format='eps')
    fig.savefig('disease_correction.png')
    print('A1: Black, A2: Red, B1: Green, B2: Blue, Centers A and B, and diseases 1 and 2.) ')

len(average_prototype_center)

In [ ]:
#center
visualizeSinglemodel(modeld,scalarId,datanoHC,labels4class,modeld.eigenvalues_,modeld.eigenvectors_,average_prototype_disease)
#disease with average disease eigen vectors


In [ ]:
visualizeSinglemodel(modelc,scalarIc,centerdata1,centerlabels1,modelc.eigenvalues_,modelc.eigenvectors_,average_prototype_center)
#disease with average disease eigen vectors
# higher negative eigen vectors on xaxis represents lower score /importance of that component

## confusion matrix 

In [ ]:
def confusionmatrix(test,predicted,labels):
    
    confusion_matrix(test, predicted, labels)
    print(len(confusion_matrix(test, predicted, labels).ravel()))
    PDPD,PDAD,ADPD,ADAD = confusion_matrix(test, predicted, labels).ravel()
            
    return PDPD,PDAD,ADPD,ADAD
            

In [ ]:
def averageconfusionmatrixd(modelmatrix,testlabelmatrix,predictedlabelmatrix):
    testlist = list(chain.from_iterable(zip(*testlabelmatrix)))
    predictedlist = list(chain.from_iterable(zip(*predictedlabelmatrix)))
    # all are averge rates
    # later might store in a matrix to index exact model
    for i in range(len(testlist)):
        PDPD,PDAD,ADPD,ADAD = 0
        npd_fpr = 0
        npd_tpr = 0
        nad_fpr = 0
        nad_tpr = 0
        PDPD,PDAD,ADPD,ADAD = confusionmatrix(testlist[i],predictedlist[i],labels=['PD','AD'])
        # did not divide by the length of the list due to reasoning added in the ppt
        npd_fpr =  (ADPD)/(ADPD+ADAD)
        npd_tpr =  (PDPD)/(PDPD+ADPD)
        nad_fpr =  (PDAD)/(PDAD+PDPD)
        nad_tpr =  (ADAD)/(ADAD+PDAD)
        
    return npd_fpr,npd_tpr,nad_fpr,nad_tpr

In [ ]:
n_fpr = {0:nad_fpr,1:npd_fpr}
n_tpr = {0:nad_tpr,1:npd_tpr}

In [ ]:
def roccurve(n_fpr,n_tpr)

In [ ]:


n_classes=["AD","PD"]
# structures
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(len(n_classes)):
    fpr[i], tpr[i], _ = roc_curve(testlabelsd[0][2],probabiltiesd[0][2][:,i],pos_label=n_classes[i], drop_intermediate=True)
    roc_auc[i] = auc(fpr[i], tpr[i])
    
figsize=(10, 10)
# roc for each class
fig, ax = plt.subplots(figsize=figsize)
ax.plot([0, 1], [0, 1], 'k--')
ax.set_xlim([0.0, 1.0])
ax.set_ylim([0.0, 1.05])
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('Receiver operating characteristic example')
for i in range(len(n_classes)):    
    ax.plot(fpr[i], tpr[i], label='ROC curve (AUC = %0.2f) for %s' % (roc_auc[i], n_classes[i]))
    ax.plot(n_fpr[i], n_tpr[i],  marker="o", markersize="12")
ax.legend(loc="best")
ax.grid(False)
fig.savefig('roc2class.png')
#ax.grid(alpha=.4)
plt.show()

In [ ]:
def transform1(X,eigenvaluesaverage,eigenvectoraverage,scale):
    eigenvaluesaverage[eigenvaluesaverage < 0] = 0
    eigenvectoraverage = np.sqrt(eigenvaluesaverage[:, None]) * eigenvectoraverage
    if scale == True:
        return np.matmul(X, eigenvectoraverage.T)
    return np.matmul(X, eigenvectoraverage.T)

In [ ]:
# removing the z transform
def visualize(model,scaler,centerdata,centerlabels,eigenvaluesaverage,eigenvectoraverage,average_prototype_center):
    centerdata = scaler.inverse_transform(centerdata)
    #transformed_data = transform1(centerdata,eigenvaluesdisease,eigenvectoraverage,True)

    #x_d = transformed_data[:, 0]
    #y_d = transformed_data[:, 1]

    # Transform the model, i.e., the prototypes (scaled by square root of eigenvalues "scale = True")
    # prototype inverser transform
    #prototypes = scaler.inverse_transform(model.prototypes_)
    
    #transformed_model = model.transform(model.prototypes_, scale=True)
    
    transformed_data = model.transform(centerdata,True)
    
    # all examples 1st and 2nd feature
    x_d = transformed_data[:, 0]
    y_d = transformed_data[:, 1]

    # Transform the model, i.e., the prototypes (scaled by square root of eigenvalues "scale = True")
    # prototype inverser transform
    prototypes = scaler.inverse_transform(average_prototype_center)

    transformed_model = model.transform(prototypes, scale=True)

    #transformed_model = transform1(prototypes,eigenvaluesaverage,eigenvectoraverage,scale=True)

    x_m = transformed_model[:, 0]
    y_m = transformed_model[:, 1]

    fig, ax = plt.subplots()
    fig.suptitle("Maindataset disease data with corresponding prototypes")
    colors = ['pink','magenta','brown']#,'pink','lightgreen']
    for i, cls in enumerate(model.classes_):
        ii = cls == centerlabels
        ax.scatter(
            x_d[ii],
            y_d[ii],
            c=colors[i],
            s=100,
            alpha=0.7,
            edgecolors="white",
            label=model.classes_[model.prototypes_labels_[i]],
        )
    ax.scatter(x_m, y_m, c=colors, s=180, alpha=0.8, edgecolors="black", linewidth=2.0)
    ax.set_xlabel("First eigenvector")
    ax.set_ylabel("Second eigenvector")
    ax.legend()
    plt.xlim([-30,30])
    ax.grid(True)
    print(model.classes_)
    #fig.savefig('visualize_center.eps', format='eps')
    fig.savefig('visualize_center.png')
    print('A1: Black, A2: Red, B1: Green, B2: Blue, Centers A and B, and diseases 1 and 2.) ')

In [ ]:
#center
visualize(centermodel[0][1],scalars1[0][1],datamatrix[0][1],labelmatrix[0][1],eigenvaluescenter,eigenvectorscenter,average_prototype_center)
#disease with average disease eigen vectors


In [ ]:
scalars1[0][1]

In [ ]:
# removing the z transform
def visualize_diseaseaverage(model,scaler,diseasedata,diseaselabel,eigenvaluesaverage,eigenvectoraverage):
    
    diseasedata = scaler.inverse_transform(diseasedata)
    transformed_data = transform1(diseasedata,eigenvaluesaverage,eigenvectoraverage,True)
    # all data points of the 1st feature
    x_d = transformed_data[:, 0]
    y_d = transformed_data[:, 1]

    # Transform the model, i.e., the prototypes (scaled by square root of eigenvalues "scale = True")
    # prototype inverser transform
    prototypes = scaler.inverse_transform(model.prototypes_)
    
    #transformed_model = model.transform(model.prototypes_, scale=True)
    transformed_model = transform1(prototypes,eigenvaluesaverage,eigenvectoraverage,True)

    print(len(model.prototypes_))
    
    # transformed model of the 1st and 2nd feature
    x_m = transformed_model[:, 0]
    y_m = transformed_model[:, 1]

    fig, ax = plt.subplots()
    fig.suptitle("Maindataset disease data with corresponding prototypes")
    #colors = ['Maroon','blue','Olive','lime','red','orange','Red','Orange','Green']
    colors = ['Maroon','blue','Olive','lime','red','orange']
    #markers = ["." , "," , "o" , "v" , "^" , "<", ">",'*','p']
    markers = ["o" , "o" , "o" , "v" , "v" , "v"]
    innerlabels = ['ADUMCG','ADUGOSM','ADCUN','PDUMCG','PDUGOSM','PDCUN']
    #for i in enumerate(model.classes_):
    for j,cls in enumerate(innerlabels):

        ii = cls == diseaselabel
        print(cls)
        ax.scatter(
        x_d[ii],
        y_d[ii],
        #c=colors[j],
        s=150,
        alpha=0.7,
        edgecolors=colors[j],
        label=innerlabels[j],
        marker = markers[j],)
    #ax.scatter(x_m, y_m, c=colors, s=180, alpha=0.8, edgecolors="black", linewidth=2.0)
    ax.set_xlabel("First eigenvector")
    ax.set_ylabel("Second eigenvector")
    ax.legend()
    ax.grid(True)
    #fig.savefig('visualize_diseaseaverage.eps', format='eps')
    fig.savefig('visualize_diseaseaverage.png')

In [ ]:
# removing the z transform
def visualize_disease(model,scaler,data,labels):
    
    data = scaler.inverse_transform(data)
    transformed_data = model.transform(data,True)
    
    # all examples 1st and 2nd feature
    x_d = transformed_data[:, 0]
    y_d = transformed_data[:, 1]

    # Transform the model, i.e., the prototypes (scaled by square root of eigenvalues "scale = True")
    # prototype inverser transform
    prototypes = scaler.inverse_transform(model.prototypes_)

    transformed_model = model.transform(prototypes, scale=True)


    
    x_m = transformed_model[:, 0]
    y_m = transformed_model[:, 1]

    fig, ax = plt.subplots()
    fig.suptitle("Maindataset disease data with corresponding prototypes")    
    #colors = ['black','Lime','gold','Teal','Navy','purple','Red','Orange','Green']
    colors = ['Maroon','blue','Olive','lime','red','orange']
    markers = ["o" , "o" , "o" , "v" , "v" , "v", "x",'x','x']
    innerlabels = ['ADUMCG','ADUGOSM','ADCUN','PDUMCG','PDUGOSM','PDCUN']
    for j,cls in enumerate(innerlabels):

        ii = cls == labels
        ax.scatter(
        x_d[ii],
        y_d[ii],
        #c=colors[j],
        s=150,
        alpha=0.7,
        edgecolors=colors[j],
        label=innerlabels[j],
        marker = markers[j])
    ax.set_xlabel("First eigenvector")
    ax.set_ylabel("Second eigenvector")
    ax.legend()
    ax.grid(True)
    #fig.savefig('visualize_individualdisease.eps', format='eps')
    fig.savefig('visualize_individualdisease.png')
    #print('A1: Black, A2: Red, B1: Green, B2: Blue, Centers A and B, and diseases 1 and 2.) ')

In [ ]:

visualize_diseaseaverage(diseasemodel[0][2],scalars2[0][2],datamatrix1[0][2],labelmatrix1[0][2],eigenvaluesdisease,eigenvectorsdisease)

# individual disease with individual labels


In [ ]:
# makes sense because correction applied to the average disease lambda, while below refers to individual model
visualize_disease(diseasemodel[0][2],scalars2[0][2],datamatrix1[0][2],labelmatrix1[0][2])

In [ ]:
# Plot the eigenvalues of the eigenvectors of the relevance matrix.
def ploteigenvalues(eigenvalues,eigenvectors,average_lambda,d):
    fig, ax = plt.subplots()
    fig.suptitle("Eigen values without correction "+d+ " main data")
    ax.bar(range(0, len(eigenvalues)), eigenvalues)
    ax.set_ylabel("Weight")
    ax.grid(False)
    fig.savefig('eigenvalues'+d+'.eps', format='eps')
    fig.savefig('eigenvalues'+d+'.png')
    #fig.savefig('eigenvalues+d+.svg", format ='svg', dpi=1200)


    # Plot the first two eigenvectors of the relevance matrix, which  is called `omega_hat`.
    fig, ax = plt.subplots()
    fig.suptitle("First Eigenvector without correction " +d+ " main data")
    ax.bar(feature_names, eigenvectors[0, :])
    ax.set_ylabel("Weight")
    ax.grid(False)
    fig.savefig('Firsteigenvector'+d+'.eps', format='eps')
    fig.savefig('Firsteigenvector'+d+'.png')
    
    fig, ax = plt.subplots()
    fig.suptitle("Second Eigenvector without correction " +d+ " main data")
    ax.bar(feature_names, eigenvectors[1, :])
    ax.set_ylabel("Weight")
    ax.grid(False)
    fig.savefig('Secondeigenvector'+d+'.eps', format='eps')
    fig.savefig('Secondeigenvector'+d+'.png')
    fig, ax = plt.subplots()
    # The relevance matrix is available after fitting the model.
    relevance_matrix = average_lambda
    fig.suptitle("Relevance Matrix Diagonal without correction " +d+ " main data")
    ax.bar(feature_names, np.diagonal(relevance_matrix))
    ax.set_ylabel("Weight")
    ax.grid(False)
    fig.savefig('relevancematrix'+d+'.eps', format='eps')
    fig.savefig('relevancematrix'+d+'.png')

In [ ]:
ploteigenvalues(eigenvaluescenter,eigenvectorscenter,average_lambda_center,'center')

In [ ]:
ploteigenvalues(modeld.eigenvalues_,modeld.eigenvectors_,modeld.lambda_,'disease')

In [ ]:
ploteigenvalues(eigenvaluesdisease,eigenvectorsdisease,average_lambda_disease,'disease')

In [ ]:
ploteigenvalues(modelc.eigenvalues_,modelc.eigenvectors_,modelc.lambda_,'disease')